In [50]:
# Helpers
import pickle
import numpy as np
import time
from collections import defaultdict
import warnings

# DB 
import psycopg2
from django.conf import settings

# Learning
from sklearn.decomposition import NMF

# Evaluation
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import roc_curve, auc, f1_score

In [2]:
connection = psycopg2.connect(user = settings.DATABASES["default"]["USER"],
                                  password = settings.DATABASES["default"]["PASSWORD"],
                                  host = settings.DATABASES["default"]["HOST"],
                                  port = settings.DATABASES["default"]["PORT"],
                                  database = settings.DATABASES["default"]["NAME"])
connection.autocommit=True
cursor = connection.cursor()

In [3]:
folder = "data/edm2020"
N_RUNS = 5

### Organizing data

In [301]:
def add_to_matrix(data, tensor, X, attempt_agg='avg', filter_attempt=False):
    attempts_count = defaultdict(int)
    for item in data:
        s_idx, q_idx, a_idx, outcome = item
        
        # If attempt filter is provided
        if filter_attempt and attempts_count[(s_idx, q_idx)] >= filter_attempt:
            continue
        else:
            attempts_count[(s_idx, q_idx)] += 1
        
        tensor[s_idx, q_idx, a_idx] = outcome
        X[s_idx, q_idx, a_idx] = 1
    tensor[np.where(X[:,:] == 0)] = None

    # # Average over all attempts for one question
    if attempt_agg == 'avg':
        # Used to ignore NaN warnings
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", category=RuntimeWarning)
            matrix = np.nanmean(tensor, axis=2)
            # 0.1 means the student at least tried opposing to 0, which is the he/she did not attempt the question
#             matrix[np.where(matrix < 0.1)] = 0
#         matrix[np.where(np.isnan(matrix) == True)] = None

    return matrix, X
    
def transform_data(data, n_attempts):
    N_STUDENTS = len(data['users_idx'])
    N_QUESTIONS = len(data['questions_idx'])
    
    max_attempt = np.max(data['train_set'][:,2])+1
    student_performance_tensor = np.zeros((N_STUDENTS, N_QUESTIONS, max_attempt))
    X = np.zeros(student_performance_tensor.shape)

    student_performance, X = add_to_matrix(data['train_set'], 
                                        student_performance_tensor, 
                                        X, filter_attempt=n_attempts)
    
    max_attempt = np.max(data['test_set'][:,2])+1
    student_performance_test_tensor = np.zeros((N_STUDENTS, N_QUESTIONS, max_attempt))
    X_test = np.zeros(student_performance_test_tensor.shape)
    student_performance_test, X_test = add_to_matrix(data['test_set'], 
                                             student_performance_test_tensor, 
                                             X_test, filter_attempt=n_attempts)
    return student_performance, X, student_performance_test, X_test

Do grid search to explore parameters

In [308]:
# %%time
n_concepts = range(18, 23)
# attempts = [False, 1, 3]
attempts = [1]
# l1_ratio = [0, 0.3, 0.5, 0.7, 1]
l1_ratio = [0, 0.3, 0.5]
# alpha = list(np.arange(0, 1, 0.1)) + list(range(1, 10, 1))
alpha = list(np.arange(0, 0.3, 0.1))

In [309]:
search = 1 * N_RUNS * len(attempts) * len(n_concepts) * len(l1_ratio) * len(alpha)
search

225

In [310]:
days = (0.4*search)/(60*60*24)
days

0.0010416666666666667

In [320]:
def run(*args):
    (dataset, data, att, student_performance, X, student_performance_test, X_test,
     del_questions, concept, alpha, l1_ratio) = args
    
    # Run NMF
    start = time.time()
    (best_student_performance_pred, best_student_knowledge, 
     best_q_matrix, best_error) = non_negative_matrix_factorization(
        student_performance, concept, alpha, l1_ratio, init=10, max_iter=1000)
#     a = non_negative_matrix_factorization(student_performance, concept, alpha, l1_ratio, init=10, max_iter=1000)
#     return a
    end = time.time()

    # Delete questions from original q_matrix
    q_matrix = np.delete(data['q_matrix'], del_questions, axis=1)
    question_similarity = cosine_similarity(q_matrix.T)
    question_hat_similarity = cosine_similarity(np.asarray(best_q_matrix).T)
    error = question_similarity - question_hat_similarity
    q_matrix_error = np.sqrt(np.sum(np.power(error,2)))
    q_matrix_rmse = np.sqrt(np.mean(np.power(error,2)))
    
    # Get train predicted values
    y = student_performance[np.where(np.isnan(student_performance) == False)]
    y_pred = best_student_performance_pred[np.where(np.isnan(student_performance) == False)]
    rmse = np.sqrt(np.mean(np.power(y - y_pred, 2))) 

    row = {
        "dataset": "run_%d" % dataset,
        "q_matrix": q_matrix,
#                         "X": X,
        "sp": student_performance,
# #                         "X_test": X_test,
        "sp_test": student_performance_test,
        "sp_hat": best_student_performance_pred,
        "sk_hat": best_student_knowledge,
        "q_matrix_hat": best_q_matrix,
#                         "mu": m,
        "concepts": concept,
        "attempts_train": int(att),
        "method": "nmf_no_imput",
        "q_matrix_error": q_matrix_error,
        "q_matrix_rmse": q_matrix_rmse,
        "reconstruction_error": best_error,
        "train_error": 0,
        "test_error": 0,
        "seconds": end-start,
        "train_rmse": rmse,
        "alpha": alpha,
        "l1_ratio": l1_ratio
    }
    
     # If just first attempt, we can binarize predictions
    if att == 1:
        # Calculate AUC and threshold to binarize prediction
        fpr, tpr, thresholds = roc_curve(y, y_pred, pos_label=1)
        auc_score = auc(fpr, tpr)
        J_stats = tpr - fpr
        J_opt_thresholds = thresholds[np.argmax(J_stats)]
        y_pred_binary = np.where(np.asarray(y_pred) > J_opt_thresholds, 1, 0)
    
         # Calculate accuracy and F1
        acc = np.logical_not(np.logical_xor(y, y_pred_binary)).sum()/len(y)
        f1 = f1_score(y, y_pred_binary)
        
        row.update({
            "train_auc": auc_score,
            "train_acc": acc,
            "auc_threshold": J_opt_thresholds,
            "train_1":  np.where(y == 1)[0].shape[0],
            "train_0":  np.where(y == 0)[0].shape[0],
            "train_pred_1": np.where(y_pred_binary == 1)[0].shape[0],
            "train_pred_0": np.where(y_pred_binary == 0)[0].shape[0],
            "train_f1": f1
        })

    # Write PSQL query
    insert_query_base = "INSERT INTO EDM2020_2020_02_19 "
    column_value = []
    insert_format = []
    query_values = []
    for col in row.keys():
        if isinstance(row[col], np.ndarray):
            query_values.append(row[col].tolist())
        else:
            query_values.append(row[col])
        column_value.append(col)
        insert_format.append("%s")

    insert_query = insert_query_base + "(" + ", ".join(column_value) + ") VALUES "
    insert_query += "(" + ", ".join(insert_format) + ")"
    query_values = tuple(query_values)
    query = cursor.mogrify(insert_query, query_values)
    cursor.execute(query)

In [312]:
args = []

for restart in range(1):
    for dataset in range(N_RUNS):
        with open("%s/run_%d.pkl" % (folder, dataset), "rb") as pklfile:
            data = pickle.load(pklfile)
        for att in attempts:
            # Get train and test data in FDTF format
            student_performance, X, student_performance_test, X_test = transform_data(data, att)
            
            # Check if questions have minimum number of attempts
            attempts_per_question = student_performance.sum(axis=0)           
            del_questions = [idx for idx,value in enumerate(attempts_per_question) if value == 0]
#             print(del_questions)
            student_performance = np.delete(student_performance, del_questions, axis=1)
            
            for l in l1_ratio:
                for a in alpha:
                    for concept in n_concepts:         
                        args.append([dataset, 
                                     data, 
                                     att, 
                                     student_performance,
                                     X,
                                     student_performance_test, 
                                     X_test,
                                     del_questions, 
                                     concept, 
                                     a, 
                                     l])

In [313]:
len(args)

225

In [408]:
N = 138

In [409]:
(dataset, data, att, student_performance, X, student_performance_test, X_test,
     del_questions, concept_a, alpha_a, l1_ratio_a) = args[N]

In [410]:
alpha_a, l1_ratio_a, concept_a

(0.0, 0, 21)

In [411]:
next_id = N

In [412]:
# %%time
connection = psycopg2.connect(user = settings.DATABASES["default"]["USER"],
                                  password = settings.DATABASES["default"]["PASSWORD"],
                                  host = settings.DATABASES["default"]["HOST"],
                                  port = settings.DATABASES["default"]["PORT"],
                                  database = settings.DATABASES["default"]["NAME"])
connection.autocommit=True
cursor = connection.cursor()

for item in args[next_id:]:
    run(*item)

RuntimeError: too many iterations

In [405]:
from scipy.optimize import nnls

def cost(A, W, H):
    from numpy import linalg
    mask = np.logical_not(np.isnan(A))
    WH = np.dot(W, H)
    WH_mask = WH[mask]
    A_mask = A[mask]
    A_WH_mask = A_mask-WH_mask
    # Since now A_WH_mask is a vector, we use L2 instead of Frobenius norm for matrix
    return linalg.norm(A_WH_mask, 2)
#     return A_WH_mask

def run_nmf(student_performance, concept, alpha, l1_ratio, init=10, max_iter=200):
    best_error = 9999
    A = student_performance.copy()
    K = concept
    W = np.abs(np.random.uniform(low=0, high=1, size=(A.shape[0], K)))
    H = np.abs(np.random.uniform(low=0, high=1, size=(K, A.shape[1])))
    W = np.divide(W, K*W.max())
    H = np.divide(H, K*H.max())
    
    for i in range(max_iter):
        if i%2 == 0:
            # Learn H, given A and W
            for j in range(A.shape[1]):
                mask_rows = np.logical_not(np.isnan(student_performance[:,j]))
                H[:,j] = nnls(W[mask_rows], A[:,j][mask_rows], maxiter=150000)[0]
        else:
            for j in range(A.shape[0]):
                mask_rows = np.logical_not(np.isnan(student_performance[j,:]))
                W[j,:] = nnls(H.transpose()[mask_rows], A[j,:][mask_rows])[0]
        WH = np.dot(W, H)
        c = cost(A, W, H)
#         return c
    return W, H, c

# def run_nmf(model, student_performance, concept, alpha, l1_ratio):
#     student_knowledge = model.fit_transform(student_performance)
#     q_matrix = model.components_
#     error = model.reconstruction_err_
    
#     return student_knowledge, q_matrix, error
    
def non_negative_matrix_factorization(student_performance, concept, alpha, l1_ratio, init=10, max_iter=200):
    
    best_error = 9999
    
#     # First init use nndsvd
#     model = NMF(n_components=concept, init='nndsvd', solver='mu', alpha=alpha, 
#                 l1_ratio=l1_ratio, max_iter=max_iter)
#     student_knowledge, q_matrix, error = run_nmf(model, student_performance, concept, alpha, l1_ratio)
    
#     if error < best_error:
#         best_student_knowledge = student_knowledge
#         best_q_matrix = q_matrix
#         best_error = error
    
    # Generate some random inits as well
    for i in range(2, init):
#         model = NMF(n_components=concept, init='random', solver='cd', 
#                     alpha=alpha, l1_ratio=l1_ratio, max_iter=max_iter)
        student_knowledge, q_matrix, error = run_nmf(student_performance, concept, alpha, l1_ratio)
#         a = run_nmf(student_performance, concept, alpha, l1_ratio)
#         return a
        
        if error < best_error:
            best_student_knowledge = student_knowledge
            best_q_matrix = q_matrix
            best_error = error
    best_student_performance_pred = np.dot(best_student_knowledge, best_q_matrix)
            
    return (best_student_performance_pred, best_student_knowledge, best_q_matrix, best_error)